# **Tutorial** - BERTopic Best Practices

Through the nature of BERTopic, its modularity, many variations of the topic modeling technique is possible. However, during the development and through the usage of the package, a set of best practices have been developed that generally lead to great results.

The following are a number of steps, parameters, and settings that you can use that will generally improve the quality of the resulting topics. In other words, after going through the quick start and getting a feeling for the API these steps should get you to the next level of performance.

**NOTE:**
    Although these are called *best practices*, it does not necessarily mean that they work across all use cases perfectly. The underlying modular nature of BERTopic is meant to take different use cases into account. After going through these practices it is advised to fine-tune wherever necessary.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="20%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic
!pip install datasets
!pip install openai

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
For this example, we will use a dataset containing abstracts and metadata from [ArXiv articles](https://huggingface.co/datasets/arxiv_dataset).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Downloading dataset
%%capture
!wget https://huggingface.co/datasets/CCRss/arXiv_dataset/resolve/main/data/train-00000-of-00008.parquet
!wget https://huggingface.co/datasets/CCRss/arXiv_dataset/resolve/main/data/train-00001-of-00008.parquet
# !wget https://huggingface.co/datasets/CCRss/arXiv_dataset/resolve/main/data/train-00002-of-00008.parquet
# !wget https://huggingface.co/datasets/CCRss/arXiv_dataset/resolve/main/data/train-00003-of-00008.parquet
# !wget https://huggingface.co/datasets/CCRss/arXiv_dataset/resolve/main/data/train-00004-of-00008.parquet
# !wget https://huggingface.co/datasets/CCRss/arXiv_dataset/resolve/main/data/train-00005-of-00008.parquet
# !wget https://huggingface.co/datasets/CCRss/arXiv_dataset/resolve/main/data/train-00006-of-00008.parquet
# !wget https://huggingface.co/datasets/CCRss/arXiv_dataset/resolve/main/data/train-00007-of-00008.parquet

In [ ]:
#@title Data
import pandas as pd

# Если нету на гугл драйв
# !wget https://huggingface.co/datasets/CCRss/arXiv_dataset/resolve/main/data/train-00000-of-00008.parquet
# df = pd.read_parquet('train-00000-of-00008.parquet')

file_list = [
    '/content/drive/MyDrive/AI/Graduation_Project/arxiv_cs_from2010to2024-01-01.parquet',
    # '/content/train-00001-of-00008.parquet',
    #'/content/drive/MyDrive/AI/Graduation_Project/dataset_vectorizer_pca/vectorized_3_merged.parquet',
    #'/content/drive/MyDrive/AI/Graduation_Project/dataset_vectorizer_pca/vectorized_4_merged.parquet',
    #'/content/drive/MyDrive/AI/Graduation_Project/dataset_vectorizer_pca/vectorized_5_merged.parquet',
    #'/content/drive/MyDrive/AI/Graduation_Project/dataset_vectorizer_pca/vectorized_6_merged.parquet',
    ## TODO fix. Что-то не так с этой частью '/content/drive/MyDrive/AI/Graduation_Project/dataset_vectorizer_pca/vectorized_7_merged.parquet',
    #'/content/drive/MyDrive/AI/Graduation_Project/dataset_vectorizer_pca/vectorized_8_merged.parquet',
]

df = pd.concat([pd.read_parquet(file_name) for file_name in file_list], ignore_index=True)
# df = pd.concat([pd.read_parquet(file_name).sample(frac=0.25) for file_name in file_list], ignore_index=True)

In [ ]:
#@title Preprocessing
# С целью удаления ненужных абстрактов.
# Создание столбца с длиной абстракта
df['abstract_length'] = df['abstract'].str.len()

# Фильтрация строк, где длина абстракта меньше 100 символов
df = df[df['abstract_length'] >= 320]

# Переиндексация DataFrame для создания непрерывной последовательности индексов
df.reset_index(drop=True, inplace=True)

# Создание столбца id_n
df['id_n'] = df.index
df

<ipython-input-2-53830a648a70>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id_n'] = df.index


,title,id,abstract,categories,doi,created,updated,authors,url,abstract_length,id_n
0,on-line viterbi algorithm and its relationship...,0704.0062,"in this paper, we introduce the on-line viterb...",cs.ds,10.1007/978-3-540-74126-8_23,2007-03-31,NaT,"['šrámek', 'brejová', 'vinař']",https://arxiv.org/abs/0704.0062,711,0
1,capacity of a multiple-antenna fading channel ...,0704.0217,given a multiple-input multiple-output (mimo) ...,cs.it math.it,10.1109/tit.2008.2011437,2007-04-02,2009-02-16,"['santipach', 'honig']",https://arxiv.org/abs/0704.0217,1641,1
2,refuting the pseudo attack on the reesse1+ cry...,0704.0492,we illustrate through example 1 and 2 that the...,cs.cr,nan,2007-04-04,2010-02-04,"['su', 'lu']",https://arxiv.org/abs/0704.0492,1345,2
3,optimal routing for decode-and-forward based c...,0704.0499,we investigate cooperative wireless relay netw...,cs.it math.it,10.1109/sahcn.2007.4292845,2007-04-04,NaT,"['ong', 'motani']",https://arxiv.org/abs/0704.0499,1164,3
4,on the kolmogorov-chaitin complexity for short...,0704.1043,a drawback of kolmogorov-chaitin complexity (k...,cs.cc cs.it math.it,nan,2007-04-08,2010-12-16,"['delahaye', 'zenil']",https://arxiv.org/abs/0704.1043,861,4
...,...,...,...,...,...,...,...,...,...,...,...
555558,on the power of random bases in fourier sampli...,quant-ph/0503114,the hidden subgroup problem (hsp) provides a u...,quant-ph cs.et,nan,2005-03-11,NaT,"['radhakrishnan', 'roetteler', 'sen']",https://arxiv.org/abs/quant-ph/0503114,1704,555558
555559,on approximately symmetric informationally com...,quant-ph/0503239,we address the problem of constructing positiv...,quant-ph cs.et,10.1063/1.1998831,2005-03-31,NaT,"['klappenecker', 'roetteler', 'shparlinski', '...",https://arxiv.org/abs/quant-ph/0503239,1119,555559
555560,limitations of quantum coset states for graph ...,quant-ph/0511148,it has been known for some time that graph iso...,quant-ph cs.et,nan,2005-11-15,NaT,"['hallgren', 'roetteler', 'sen']",https://arxiv.org/abs/quant-ph/0511148,1196,555560
555561,computing local invariants of qubit systems,quant-ph/9712040,we investigate means to describe the non-local...,quant-ph cs.et,10.1103/physreva.58.1833,1997-12-18,1998-05-29,"['grassl', 'roetteler', 'beth']",https://arxiv.org/abs/quant-ph/9712040,788,555561


In [ ]:
import pandas as pd

# Create a sample DataFrame
# df = pd.DataFrame({
#     'abstract': ['This is not null', None, 'Another non-null value', 'Yet another null value']
# })

# Check for null values in the 'abstract' column
null_check = df['abstract'].isnull().any()

# Print the result
print('В столбце "abstract" есть null значения:', null_check)

В столбце "abstract" есть null значения: False


In [ ]:
# from datasets import load_dataset

dataset = df

# Extract abstracts to train on and corresponding titles
abstracts = dataset["abstract"]
titles = dataset["title"]

In [ ]:
abstracts[0]

'in this paper, we introduce the on-line viterbi algorithm for decoding hiddenmarkov models (hmms) in much smaller than linear space. our analysis ontwo-state hmms suggests that the expected maximum memory used to decodesequence of length $n$ with $m$-state hmm can be as low as $\\theta(m\\log n)$,without a significant slow-down compared to the classical viterbi algorithm.classical viterbi algorithm requires $o(mn)$ space, which is impractical foranalysis of long dna sequences (such as complete human genome chromosomes) andfor continuous data streams. we also experimentally demonstrate the performanceof the on-line viterbi algorithm on a simple hmm for gene finding on bothsimulated and real dna sequences.'

**🔥 Tip - Sentence Splitter 🔥**
***
 Whenever you have large documents, you typically want to split them up into either paragraphs or sentences. A nice way to do so is by using NLTK's sentence splitter which is nothing more than:

```python
from nltk.tokenize import sent_tokenize, word_tokenize
sentences = [sent_tokenize(abstract) for abstract in abstracts]
sentences = [sentence for doc in sentences for sentence in doc]
```

***

# **Best Practices**

With feedback from the community throughout the development of BERTopic and the core maintainer's personal experience, there are a number of best practices developed that generally lead to an improved topic model.

The goal of these best practices to quickly guide the user to what is commonly used to speed-up training, improve performance, explore alternatives, etc. Instead of having to search through many issues and discussions, an overview of best practices are discussed here.

To start off, it is important to have a general idea of the pipeline of BERTopic as it relates to many of these best practices.

BERTopic can be viewed as a sequence of steps to create its topic representations. There are five steps to this process:

![https://maartengr.github.io/BERTopic/algorithm/default.svg](https://maartengr.github.io/BERTopic/algorithm/default.svg)

The pipeline above implies significant modularity of BERTopic. Each step in this process was carefully selected such that they are all somewhat independent from one another.

As a result, we can adopt the pipeline to the current state-of-the-art with respect to each individual step:

 ![https://maartengr.github.io/BERTopic/algorithm/modularity.svg](https://maartengr.github.io/BERTopic/algorithm/modularity.svg)

## **Pre-calculate Embeddings**
After having created our data, namely `abstracts`, we can dive into the very first best practice, **pre-calculating embeddings**.

BERTopic works by converting documents into numerical values, called embeddings. This process can be very costly, especially if we want to iterate over parameters. Instead, we can calculate those embeddings once and feed them to BERTopic to skip calculating embeddings each time.

In [ ]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(abstracts, show_progress_bar=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/17362 [00:00<?, ?it/s]

## **Preventing Stochastic Behavior**
In BERTopic, we generally use a dimensionality reduction algorithm to reduce the size of the embeddings. This is done to prevent the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) to a certain degree.

As a default, this is done with [UMAP](https://github.com/lmcinnes/umap) which is an incredible algorithm for reducing dimensional space. However, by default, it shows stochastic behavior which creates different results each time you run it. To prevent that, we will need to set a `random_state` of the model before passing it to BERTopic.

As a result, we can now fully reproduce the results each time we run the model.

In [ ]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

## **Controlling Number of Topics**
There is a parameter to control the number of topics, namely `nr_topics`. This parameter, however, merges topics **after** they have been created. It is a parameter that supports creating a fixed number of topics.

However, it is advised to control the number of topics through the cluster model which is by default HDBSCAN. HDBSCAN has a parameter, namely `min_topic_size` that indirectly controls the number of topics that will be created.

A higher `min_topic_size` will generate fewer topics and a lower `min_topic_size` will generate more topics.

Here, we will go with `min_topic_size=40` to get around XXX topics.

In [ ]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

## **Improving Default Representation**
The default representation of topics is calculated through [c-TF-IDF](https://maartengr.github.io/BERTopic/algorithm/algorithm.html#5-topic-representation). However, c-TF-IDF is powered by the [CountVectorizer](https://maartengr.github.io/BERTopic/getting_started/vectorizers/vectorizers.html) which converts text into tokens. Using the CountVectorizer, we can do a number of things:

* Remove stopwords
* Ignore infrequent words
* Increase

In other words, we can preprocess the topic representations **after** documents are assigned to topics. This will not influence the clustering process in any way.

Here, we will ignore English stopwords and infrequent words. Moreover, by increasing the n-gram range we will consider topic representations that are made up of one or two words.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

## **Additional Representations**
Previously, we have tuned the default representation but there are quite a number of [other topic representations](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html) in BERTopic that we can choose from. From [KeyBERTInspired](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#keybertinspired) and [PartOfSpeech](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#partofspeech), to [OpenAI's ChatGPT](https://maartengr.github.io/BERTopic/getting_started/representation/llm.html#chatgpt) and [open-source](https://maartengr.github.io/BERTopic/getting_started/representation/llm.html#langchain) alternatives, many representations are possible.

In BERTopic, you can model many different topic representations simultanously to test them out and get different perspectives of topic descriptions. This is called [multi-aspect](https://maartengr.github.io/BERTopic/getting_started/multiaspect/multiaspect.html) topic modeling.

Here, we will demonstrate a number of interesting and useful representations in BERTopic:

* KeyBERTInspired
  * A method that derives inspiration from how KeyBERT works
* PartOfSpeech
  * Using SpaCy's POS tagging to extract words
* MaximalMarginalRelevance
  * Diversify the topic words
* OpenAI
  * Use ChatGPT to label our topics


In [ ]:
import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
client = openai.OpenAI(api_key="sk-...")
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    "POS": pos_model
}

## **Training**
Now that we have a set of best practices, we can use them in our training loop. Here, several different representations, keywords and labels for our topics will be created. If you want to iterate over the topic model it is advised to use the pre-calculated embeddings as that significantly speeds up training.

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

topics, probs = topic_model.fit_transform(abstracts, embeddings)

2024-02-02 06:40:20,723 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-02 06:53:15,288 - BERTopic - Dimensionality - Completed ✓
2024-02-02 06:53:15,299 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-02 06:54:22,373 - BERTopic - Cluster - Completed ✓
2024-02-02 06:54:22,491 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-02 07:02:40,011 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,POS,Representative_Docs
0,-1,242748,-1_data_based_model_paper,"[data, based, model, paper, models, using, res...","[learning, deep, neural, trained, training, ta...","[data, based, model, paper, models, using, res...","[data, model, paper, models, results, learning...",[audio-visual segmentation (avs) is a complex ...
1,0,9242,0_numerical_element_equations_equation,"[numerical, element, equations, equation, fini...","[discontinuous galerkin, galerkin, discretizat...","[numerical, element, equations, equation, fini...","[numerical, element, equations, equation, fini...",[we derive well-posed boundary conditions for ...
2,1,8136,1_graphs_graph_vertex_vertices,"[graphs, graph, vertex, vertices, edge, edges,...","[planar graphs, chromatic, free graphs, subgra...","[graphs, graph, vertex, vertices, edge, edges,...","[graphs, graph, vertex, vertices, edge, edges,...",[a graph $g$ is $k$-vertex-critical if $g$ has...
3,2,8055,2_quantum_classical_qubits_circuits,"[quantum, classical, qubits, circuits, qubit, ...","[quantum algorithms, quantum computing, quantu...","[quantum, classical, qubits, circuits, qubit, ...","[quantum, classical, qubits, circuits, qubit, ...",[quantum machine learning has proven to be a f...
4,3,8024,3_power_energy_grid_voltage,"[power, energy, grid, voltage, load, electrici...","[power grid, smart grid, power grids, microgri...","[power, energy, grid, voltage, load, electrici...","[power, energy, grid, voltage, load, electrici...",[cascading failures are one of the main reason...
...,...,...,...,...,...,...,...,...
382,381,154,381_rbm_boltzmann_rbms_restricted boltzmann,"[rbm, boltzmann, rbms, restricted boltzmann, b...","[restricted boltzmann, restricted boltzmannmac...","[rbm, boltzmann, rbms, restricted boltzmann, b...","[rbm, boltzmann, rbms, restricted boltzmann, b...",[we investigate the potential of a restricted ...
383,382,153,382_testing_distribution_distributions_sample,"[testing, distribution, distributions, sample,...","[distribution testing, uniformity testing, sam...","[testing, distribution, distributions, sample,...","[testing, distribution, distributions, sample,...",[we study the problem of testing discrete dist...
384,383,153,383_wavelet_framelets_wavelets_transform,"[wavelet, framelets, wavelets, transform, tigh...","[multi wavelets, framelets, wavelets, tight fr...","[wavelet, framelets, wavelets, transform, tigh...","[wavelet, framelets, wavelets, transform, tigh...",[construction of multivariate tight framelets ...
385,384,152,384_multimodal_modalities_modality_modal,"[multimodal, modalities, modality, modal, fusi...","[multimodal learning, multimodallearning, lear...","[multimodal, modalities, modality, modal, fusi...","[multimodal, modalities, modality, modal, fusi...",[multimodal learning aims to imitate human bei...


To get all representations for a single topic, we simply run the following:

In [ ]:
topic_model.get_topic(1, full=True)

{'Main': [('graphs', 0.021768386163764513),
  ('graph', 0.01726014032524179),
  ('vertex', 0.015354878827349533),
  ('vertices', 0.013660837167721405),
  ('edge', 0.009214079257728606),
  ('edges', 0.008992450550181668),
  ('coloring', 0.008811357423711875),
  ('planar', 0.008481578757236643),
  ('log', 0.008134249224069539),
  ('delta', 0.006894240417085174)],
 'KeyBERT': [('planar graphs', 0.53053766),
  ('chromatic', 0.52107173),
  ('free graphs', 0.508548),
  ('subgraphs', 0.48813796),
  ('graphs', 0.48181364),
  ('coloring', 0.4698238),
  ('vertex', 0.46417993),
  ('vertex set', 0.45672962),
  ('vertex cover', 0.45515475),
  ('subgraph', 0.45013025)],
 'MMR': [('graphs', 0.021768386163764513),
  ('graph', 0.01726014032524179),
  ('vertex', 0.015354878827349533),
  ('vertices', 0.013660837167721405),
  ('edge', 0.009214079257728606),
  ('edges', 0.008992450550181668),
  ('coloring', 0.008811357423711875),
  ('planar', 0.008481578757236643),
  ('log', 0.008134249224069539),
  ('delt

**NOTE**: The labels generated by OpenAI's **ChatGPT** are especially interesting to use throughout your model. Below, we will go into more detail how to set that as a custom label.

**🔥 Tip - Parameters 🔥**
***
If you would like to return the topic-document probability matrix, then it is advised to use `calculate_probabilities=True`. Do note that this can significantly slow down training. To speed it up, use [cuML's HDBSCAN](https://maartengr.github.io/BERTopic/getting_started/clustering/clustering.html#cuml-hdbscan) instead. You could also approximate the topic-document probability matrix with `.approximate_distribution` which will be discussed later.
***

## **(Custom) Labels**
The default label of each topic are the top 3 words in each topic combined with an underscore between them.

This, of course, might not be the best label that you can think of for a certain topic. Instead, we can use `.set_topic_labels` to manually label all or certain topics.

We can also use `.set_topic_labels` to use one of the other topic representations that we had before, like `KeyBERTInspired` or even `OpenAI`.

In [ ]:
# Label the topics yourself
# topic_model.set_topic_labels({1: "Space Travel", 7: "Religion"})

# or use one of the other topic representations, like KeyBERTInspired
keybert_topic_labels = {topic: " | ".join(list(zip(*values))[0][:3]) for topic, values in topic_model.topic_aspects_["KeyBERT"].items()}
topic_model.set_topic_labels(keybert_topic_labels)

# or ChatGPT's labels
# chatgpt_topic_labels = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["OpenAI"].items()}
# chatgpt_topic_labels[-1] = "Outlier Topic"
# topic_model.set_topic_labels(chatgpt_topic_labels)

Now that we have set the updated topic labels, we can access them with the many functions used throughout BERTopic. Most notably, you can show the updated labels in visualizations with the `custom_labels=True` parameters.

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,CustomName,Representation,KeyBERT,MMR,POS,Representative_Docs
0,-1,242748,-1_data_based_model_paper,learning | deep | neural,"[data, based, model, paper, models, using, res...","[learning, deep, neural, trained, training, ta...","[data, based, model, paper, models, using, res...","[data, model, paper, models, results, learning...",[audio-visual segmentation (avs) is a complex ...
1,0,9242,0_numerical_element_equations_equation,discontinuous galerkin | galerkin | discretiza...,"[numerical, element, equations, equation, fini...","[discontinuous galerkin, galerkin, discretizat...","[numerical, element, equations, equation, fini...","[numerical, element, equations, equation, fini...",[we derive well-posed boundary conditions for ...
2,1,8136,1_graphs_graph_vertex_vertices,planar graphs | chromatic | free graphs,"[graphs, graph, vertex, vertices, edge, edges,...","[planar graphs, chromatic, free graphs, subgra...","[graphs, graph, vertex, vertices, edge, edges,...","[graphs, graph, vertex, vertices, edge, edges,...",[a graph $g$ is $k$-vertex-critical if $g$ has...
3,2,8055,2_quantum_classical_qubits_circuits,quantum algorithms | quantum computing | quant...,"[quantum, classical, qubits, circuits, qubit, ...","[quantum algorithms, quantum computing, quantu...","[quantum, classical, qubits, circuits, qubit, ...","[quantum, classical, qubits, circuits, qubit, ...",[quantum machine learning has proven to be a f...
4,3,8024,3_power_energy_grid_voltage,power grid | smart grid | power grids,"[power, energy, grid, voltage, load, electrici...","[power grid, smart grid, power grids, microgri...","[power, energy, grid, voltage, load, electrici...","[power, energy, grid, voltage, load, electrici...",[cascading failures are one of the main reason...
...,...,...,...,...,...,...,...,...,...
382,381,154,381_rbm_boltzmann_rbms_restricted boltzmann,restricted boltzmann | restricted boltzmannmac...,"[rbm, boltzmann, rbms, restricted boltzmann, b...","[restricted boltzmann, restricted boltzmannmac...","[rbm, boltzmann, rbms, restricted boltzmann, b...","[rbm, boltzmann, rbms, restricted boltzmann, b...",[we investigate the potential of a restricted ...
383,382,153,382_testing_distribution_distributions_sample,distribution testing | uniformity testing | sa...,"[testing, distribution, distributions, sample,...","[distribution testing, uniformity testing, sam...","[testing, distribution, distributions, sample,...","[testing, distribution, distributions, sample,...",[we study the problem of testing discrete dist...
384,383,153,383_wavelet_framelets_wavelets_transform,multi wavelets | framelets | wavelets,"[wavelet, framelets, wavelets, transform, tigh...","[multi wavelets, framelets, wavelets, tight fr...","[wavelet, framelets, wavelets, transform, tigh...","[wavelet, framelets, wavelets, transform, tigh...",[construction of multivariate tight framelets ...
385,384,152,384_multimodal_modalities_modality_modal,multimodal learning | multimodallearning | lea...,"[multimodal, modalities, modality, modal, fusi...","[multimodal learning, multimodallearning, lear...","[multimodal, modalities, modality, modal, fusi...","[multimodal, modalities, modality, modal, fusi...",[multimodal learning aims to imitate human bei...


Notice that the overview in `.get_topic_info` now also includes the column `CustomName`. That is the custom label that we just created for each topic.

## **Topic-Document Distribution**
If using `calculate_probabilities=True` is not possible, than you can [approximate the topic-document distributions](https://maartengr.github.io/BERTopic/getting_started/distribution/distribution.html) using `.approximate_distribution`. It is a fast and flexible method for creating different topic-document distributions.

In [ ]:
# `topic_distr` contains the distribution of topics in each document
topic_distr, _ = topic_model.approximate_distribution(abstracts, window=8, stride=4)

100%|██████████| 556/556 [24:58<00:00,  2.70s/it]


Next, lets take a look at a specific abstract and see how the topic distribution was extracted:

In [ ]:
abstract_id = 10
print(abstracts[abstract_id])

  Serre obtained the p-adic limit of the integral Fourier coefficient of
modular forms on $SL_2(\mathbb{Z})$ for $p=2,3,5,7$. In this paper, we extend
the result of Serre to weakly holomorphic modular forms of half integral weight
on $\Gamma_{0}(4N)$ for $N=1,2,4$. A proof is based on linear relations among
Fourier coefficients of modular forms of half integral weight. As applications
we obtain congruences of Borcherds exponents, congruences of quotient of
Eisentein series and congruences of values of $L$-functions at a certain point
are also studied. Furthermore, the congruences of the Fourier coefficients of
Siegel modular forms on Maass Space are obtained using Ikeda lifting.



In [ ]:
# Visualize the topic-document distribution for a single document
topic_model.visualize_distribution(topic_distr[abstract_id])

In [ ]:
# Visualize the topic-document distribution for a single document
topic_model.visualize_distribution(topic_distr[abstract_id], custom_labels=True)

It seems to have extracted a number of topics that are relevant and shows the distributions of these topics across the abstract. We can go one step further and visualize them on a token-level:

In [ ]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(abstracts[abstract_id], calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(abstracts[abstract_id], topic_token_distr[0])
df

100%|██████████| 1/1 [00:00<00:00,  1.63it/s]


,Serre,obtained,the,adic,limit,of,the,integral,Fourier,coefficient,of,modular,forms,on,SL_2,mathbb,for,In,this,paper,we,extend,the,result,of,Serre,to,weakly,holomorphic,modular,forms,of,half,integral,weight,on,Gamma_,4N,for,proof,is,based,on,linear,relations,among,Fourier,coefficients,of,modular,forms,of,half,integral,weight,As,applications,we,obtain,congruences,of,Borcherds,exponents,congruences,of,quotient,of,Eisentein,series,and,congruences,of,values,of,functions,at,certain,point,are,also,studied,Furthermore,the,congruences,of,the,Fourier,coefficients,of,Siegel,modular,forms,on,Maass,Space,are,obtained,using,Ikeda,lifting
21_modular_galois_adic_forms,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.173,0.488,0.662,0.662,0.489,0.175,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.118,0.323,0.560,0.769,0.651,0.446,0.209,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.121,0.329,0.644,0.852,0.731,0.523,0.209,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.119,0.369,0.618,0.801,0.682,0.432,0.183,0.000,0.000,0.000,0.000,0.000,0.000
37_omega_solutions_elliptic_boundary,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.141,0.141,0.141,0.141,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
64_hardy_spaces_infty_mathbb,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.181,0.181,0.181,0.181,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
93_zeta_zeta function_riemann_riemann zeta,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.137,0.137,0.137,0.137,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
242_holomorphic_cr_stein_pseudoconvex,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.120,0.225,0.325,0.449,0.329,0.224,0.124,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


**🔥 Tip - `use_embedding_model` 🔥**
***
As a default, we compare the c-TF-IDF calculations between the token sets and all topics. Due to its bag-of-word representation, this is quite fast. However, you might want to use the selected embedding_model instead to do this comparison. Do note that due to the many token sets, it is often computationally quite a bit slower:

```python
topic_distr, _ = topic_model.approximate_distribution(docs, use_embedding_model=True)
```
***




## **Outlier Reduction**
By default, HDBSCAN generates outliers which is a helpful mechanic in creating accurate topic representations. However, you might want to assign every single document to a topic. We can use `.reduce_outliers` to map some or all outliers to a topic:

In [ ]:
# Reduce outliers
new_topics = topic_model.reduce_outliers(abstracts, topics)

# Reduce outliers with pre-calculate embeddings instead
new_topics = topic_model.reduce_outliers(abstracts, topics, strategy="embeddings", embeddings=embeddings)

100%|██████████| 272/272 [13:41<00:00,  3.02s/it]


**💡  NOTE - Update Topics with Outlier Reduction 💡**
***
After having generated updated topic assignments, we can pass them to BERTopic in order to update the topic representations:

```python
topic_model.update_topics(docs, topics=new_topics)
```

It is important to realize that updating the topics this way may lead to errors if topic reduction or topic merging techniques are used afterwards. The reason for this is that when you assign a -1 document to topic 1 and another -1 document to topic 2, it is unclear how you map the -1 documents. Is it matched to topic 1 or 2.
***

## **Visualize Topics**

With visualizations, we are closing into the realm of subjective "best practices". These are things that I generally do because I like the representations but your experience might differ.

Having said that, there are two visualizations that are my go-to when visualizing the topics themselves:

* `topic_model.visualize_topics()`
* `topic_model.visualize_hierarchy()`

In [ ]:
topic_model.visualize_topics(custom_labels=True)

In [ ]:
topic_model.visualize_topics(custom_labels=True)

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(abstracts)

100%|██████████| 276/276 [00:20<00:00, 13.52it/s]


In [ ]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

[119, 103], [81, 116], [241, 235], [247, 49], [193, 72], [135, 222],

In [ ]:
topics_to_merge = [[119, 103], [81, 116], [241, 235], [247, 49], [193, 72], [135, 222]]
topic_model.merge_topics(abstracts, topics_to_merge)

In [ ]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
topic_model.visualize_hierarchy(custom_labels=True)

In [ ]:
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

## **Visualize Documents**

When visualizing documents, it helps to have embedded the documents beforehand to speed up computation. Fortunately, we have already done that as a "best practice".

Visualizing documents in 2-dimensional space helps in understanding the underlying structure of the documents and topics.

In [ ]:
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

KeyboardInterrupt: 

The following plot is **interactive** which means that you can zoom in, double click on a label to only see that one and generally interact with the plot:

In [ ]:
# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
# NOTE: You can hide the hover with `hide_document_hover=True` which is especially helpful if you have a large dataset
topic_model.visualize_documents(titles, reduced_embeddings=reduced_embeddings, custom_labels=True)

In [ ]:
# We can also hide the annotation to have a more clear overview of the topics
topic_model.visualize_documents(titles, reduced_embeddings=reduced_embeddings, custom_labels=True, hide_annotations=True)

**💡  NOTE - 2-dimensional space 💡**
***
Although visualizing the documents in 2-dimensional gives an idea of their underlying structure, there is a risk involved.

Visualizing the documents in 2-dimensional space means that we have lost significant information since the original embeddings were more than 384 dimensions. Condensing all that information in 2 dimensions is simply not possible. In other words, it is merely an **approximation**, albeit quite an accurate one.
***

## **Serialization**

When saving a BERTopic model, there are several ways in doing so. You can either save the entire model with `pickle`, `pytorch`, or `safetensors`.

Personally, I would advise going with `safetensors` whenever possible. The reason for this is that the format allows for a very small topic model to be saved and shared.

When saving a model with `safetensors`, it skips over saving the dimensionality reduction and clustering models. The `.transform` function will still work without these models but instead assign topics based on the similarity between document embeddings and the topic embeddings.

As a result, the `.transform` step might give different results but it is generally worth it considering the smaller and significantly faster model.

In [ ]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("/content/my_model_dir2", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

As mentioned above, loading can be done as follows:

In [ ]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
# Define embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load model and add embedding model
topic_model = BERTopic.load("/content/my_model_dir2", embedding_model=embedding_model)

# Huggingface


In [ ]:
!cp -r /content/BERTopic_555abstracts_cs_from2010to2024-01-01 "/content/drive/MyDrive/AI/BERT"


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from huggingface_hub import HfApi
import os

api = HfApi()
folder_path = "/content/my_model_dir"
repo_id = "CCRss/top2vec_science_abstracts"
folder_in_repo = "BERTopic_model/"  # Папка в репозитории

# Пройти по всем файлам в папке и загрузить каждый файл
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        api.upload_file(
            path_or_fileobj=file_path,
            path_in_repo=folder_in_repo + filename,  # Добавление названия папки к пути файла в репозитории
            repo_id=repo_id,
            repo_type="model"
        )